# 深層学習- 1
参考：  
* [Udemy講座：「①米国AI開発者がやさしく教える深層学習超入門第一弾【Pythonで実践】」](https://www.udemy.com/course/deeplearning1/learn/lecture/40143418)
* https://axa.biopapyrus.jp/deep-learning/
* [パーセプトロン](https://axa.biopapyrus.jp/deep-learning/perceptron.html)

## 深層学習とは
深層学習は機械学習のアルゴリズムの一種である。  
従来の機械学習手法（ロジスティック回帰やGBDTなど）は人が特徴量を作ってモデルに渡して学習させていたため、  
良い特徴量をいかに作るかが重要であった。  
一方、深層学習では入力データをモデルに渡すとモデル自身が特徴量を作って学習するため、  
良い特徴量を作ってくれるモデルをいかに作るかが重要という点で従来の機械学習手法と異なる。  
深層学習ではニューラルネットワーク(NN)という人間の神経を模したモデルを用いてデータを学習する。  
このNNの層を深くしていくことで精度が高いモデル構築することが出来る。  

深層学習の適用例は下記。  
昨今の生成AIも深層学習の応用であり、社会に非常に大きな影響を与えている技術の一つ。
* 音声認識
* 医用画像診断
* 顔認証
* 株価予想
* 画像生成

## Pytorch
深層学習ライブラリとしてはよくPytorchが用いられる。  
Tensorflowも一時期使われていたが、最近はPytorchが主流。  
深層学習に用いるデータはPytorchのTensorモジュールを使って定義する。  